In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import random

#3.1

In [2]:
df = pd.read_csv('philly_users_businesses_stars.csv')
old_user_shape = df.shape
old_business_shape = df.shape
new_user_shape =(0,0)
new_business_shape = (0,0)
print(old_business_shape,old_user_shape)

while((old_user_shape!=new_user_shape) or (old_business_shape!=new_business_shape)):
    user_counts = df.groupby('user_id')['business_id'].count()
    users_to_keep = user_counts[user_counts > 30]
    old_user_shape = new_user_shape
    new_user_shape = users_to_keep.shape
    df = df.loc[df['user_id'].isin(users_to_keep.index)]


    business_counts = df.groupby('business_id')['user_id'].count()
    businesses_to_keep = business_counts[business_counts > 50]
    old_business_shape = new_business_shape
    new_business_shape = businesses_to_keep.shape
    df = df.loc[df['business_id'].isin(businesses_to_keep.index)]
    print("prune")

print(df)

chunks = pd.read_json('yelp_academic_dataset_user.json', lines=True,chunksize=100000)
df_users = pd.DataFrame()
dataProcessed = 0
for chunk in chunks:
    dataProcessed+=1
    #temp = pd.merge(left=df, right=chunk, on='user_id')
    temp = chunk.loc[chunk['user_id'].isin(df['user_id'])]
    print(f'data processed = {dataProcessed*100000}')
    df_users = pd.concat([df_users,temp])

print(df_users.shape)

(967552, 3) (967552, 3)
prune
prune
prune
prune
prune
prune
prune
prune
prune
prune
                       user_id             business_id  stars
3       smOvOajNG0lS4Pq7d8g4JQ  RZtGWDLCAtuipwaZ-UfjmQ    4.0
5       IQsF3Rc6IgCzjVV9DE8KXg  eFvzHawVJofxSnD7TgbZtg    5.0
10      ZGjgfSvjQK886kiTzLwfLQ  EtKSTHV5Qx_Q7Aur9o4kQQ    5.0
11      IKbjLnfBQtEyVzEu8CuOLg  VJEzpfLs_Jnzgqh5A_FVTg    4.0
13      NUtIAX-ygn474tDg5nmesg  6LCZLGa09Qifn6rG7-DNrg    4.0
...                        ...                     ...    ...
967542  bJ5FtCtZX3ZZacz2_2PJjA  wMQkdK2aNMvq2xoojC98Mw    4.0
967543  bJ5FtCtZX3ZZacz2_2PJjA  SOsjW1JARmtHUFtpFlp8rw    4.0
967545  LHWtjTG7e1NzNPYUbUo-9w  rgeuy1qbw6Z8B6CSVANHIA    5.0
967550  7ziWZULyiZv2TesYNMFf4g  qQO7ErS_RAN4Vs1uX0L55Q    4.0
967551  MCzlzlOw7IGbRAKVjJBPtg  fcGexL5VH5G2Xw0tRj9uOQ    3.0

[101923 rows x 3 columns]
data processed = 100000
data processed = 200000
data processed = 300000
data processed = 400000
data processed = 500000
data processed = 600000
d

In [3]:

#df_unique_users = df_users.drop_duplicates(subset=['user_id'],keep='first', inplace=False)
#print(df_unique_users.shape)
print(df.columns)
G = nx.Graph()
unique_user_ids = df_users['user_id'].unique()
G.add_nodes_from(unique_user_ids)
print(len(G.nodes()))

df_users['friends'] = df_users['friends'].apply(lambda x: ', '.join([friend for friend in x.split(', ') if friend in unique_user_ids]))
print(df_users.columns)
for index, row in df_users.iterrows():
    user_id = str(row['user_id'])
    lst_friends = row['friends'].split(', ')
    tuple_list = [(user_id,friend) for friend in lst_friends]
    G.add_edges_from(tuple_list)

largest_cc = max(nx.connected_components(G), key=len)
CC_max = nx.subgraph(G,largest_cc)
print(len(CC_max.nodes()))


Index(['user_id', 'business_id', 'stars'], dtype='object')
1604
Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')
1504


#3.2

In [4]:
selected_user_ids = random.sample(list(unique_user_ids), 100)
selected_reviews = df[df['user_id'].isin(selected_user_ids)]
business_ids = selected_reviews['business_id'].unique()
selected_business_ids = random.sample(list(business_ids),100)

Dtest = selected_reviews[selected_reviews['business_id'].isin(selected_business_ids)]
print(Dtest)
merged = pd.merge(df,Dtest, on=['user_id','business_id','stars'],how='outer',indicator=True)
print(merged)
merged = merged.loc[merged['_merge']=='left_only']
print(merged)
Dtrain = merged.drop('_merge',axis=1)
print(Dtrain)


                       user_id             business_id  stars
837     4_IJKLhoyof73pvRal4Exw  EtKSTHV5Qx_Q7Aur9o4kQQ    4.0
1877    03F1c27jkAikld3z5tSdpQ  u7_3L1NBWgxhBM_B-cmmnA    5.0
11743   zrESudirjip-s8wxxAB0xw  EtKSTHV5Qx_Q7Aur9o4kQQ    5.0
13533   zrESudirjip-s8wxxAB0xw  4_W5pstoN1166TGjjPOrMg    4.0
14002   y3Wtx1pOvTiqsJRkjceXiw  wUnLSg_GKfEIQ5CQQ770_g    4.0
...                        ...                     ...    ...
951928  V-Tv_2hv1Ag3wQrqOpVnEQ  SOsjW1JARmtHUFtpFlp8rw    4.0
961789  Ff8hG_00h6SO5unNTXPFQQ  jkGQQ4_LgJx3hwPtCFkzbQ    4.0
962345  V-Tv_2hv1Ag3wQrqOpVnEQ  SOsjW1JARmtHUFtpFlp8rw    5.0
962916  efoFHUP08msKW73LPkZxWA  jkGQQ4_LgJx3hwPtCFkzbQ    1.0
967123  V-Tv_2hv1Ag3wQrqOpVnEQ  SOsjW1JARmtHUFtpFlp8rw    1.0

[733 rows x 3 columns]
                       user_id             business_id  stars     _merge
0       smOvOajNG0lS4Pq7d8g4JQ  RZtGWDLCAtuipwaZ-UfjmQ    4.0  left_only
1       IQsF3Rc6IgCzjVV9DE8KXg  eFvzHawVJofxSnD7TgbZtg    5.0  left_only
2       ZGjgf

#3.3

In [5]:
Dtrain = pd.read_csv('train_data.csv')
Dtest = pd.read_csv('test_data.csv')
#print(Dtest)
Btest = Dtest[Dtest['business_id'].isin(selected_business_ids)]
print(Dtrain)


for row in Dtrain.iterrows():
    u=row[1]['user_id']
    b=row[1]['business_id']
    r=row[1]['stars']
    

       Unnamed: 0                 user_id             business_id  stars
0               3  smOvOajNG0lS4Pq7d8g4JQ  RZtGWDLCAtuipwaZ-UfjmQ    4.0
1               5  IQsF3Rc6IgCzjVV9DE8KXg  eFvzHawVJofxSnD7TgbZtg    5.0
2              10  ZGjgfSvjQK886kiTzLwfLQ  EtKSTHV5Qx_Q7Aur9o4kQQ    5.0
3              11  IKbjLnfBQtEyVzEu8CuOLg  VJEzpfLs_Jnzgqh5A_FVTg    4.0
4              13  NUtIAX-ygn474tDg5nmesg  6LCZLGa09Qifn6rG7-DNrg    4.0
...           ...                     ...                     ...    ...
96589      967542  bJ5FtCtZX3ZZacz2_2PJjA  wMQkdK2aNMvq2xoojC98Mw    4.0
96590      967543  bJ5FtCtZX3ZZacz2_2PJjA  SOsjW1JARmtHUFtpFlp8rw    4.0
96591      967545  LHWtjTG7e1NzNPYUbUo-9w  rgeuy1qbw6Z8B6CSVANHIA    5.0
96592      967550  7ziWZULyiZv2TesYNMFf4g  qQO7ErS_RAN4Vs1uX0L55Q    4.0
96593      967551  MCzlzlOw7IGbRAKVjJBPtg  fcGexL5VH5G2Xw0tRj9uOQ    3.0

[96594 rows x 4 columns]
